# MLflow Results Analysis

This notebook fetches all trials from a given MLflow experiment for analysis.

## 1. Setup and Connection

In [ ]:
import warnings

import mlflow
import pandas as pd
import pydash as pyd
import seaborn as sns
from mlflow import MlflowClient

warnings.filterwarnings("ignore")

# MLflow configuration
TRACKING_URI = "http://127.0.0.1:5001"
# EXPERIMENT_ID = "413909140794424369"  # graph-clf, batch norm
EXPERIMENT_ID = "740523204863010596"  # graph-clf, hierarchical pooling layer

DESIGN_CHOICE = "HPOOLING"

print(f"Connecting to MLflow server at: {TRACKING_URI}")
print(f"Target experiment ID: {EXPERIMENT_ID}")

print(f"DESIGN_CHOICE={DESIGN_CHOICE}")
if DESIGN_CHOICE == "BN":
    design_choic_name = "use_batchnorm"
    design_choice_data_path = "data.params.model/post_mp_layer/use_batchnorm"

    metric_name = "test_roc_auc"
    metric_data_path = "data.metrics.test_roc_auc"
elif DESIGN_CHOICE == "HPOOLING":
    design_choic_name = "hpooling"
    design_choice_data_path = "data.params.model/mp_layers/0/pooling/type"

    metric_name = "test_roc_auc"
    metric_data_path = "data.metrics.test_roc_auc"

In [ ]:
# Set tracking URI and create client
mlflow.set_tracking_uri(TRACKING_URI)
client = MlflowClient()

# Validate experiment exists
try:
    experiment = client.get_experiment(EXPERIMENT_ID)
    print(f"✓ Connected to experiment: {experiment.name}")
    print(f"  Experiment ID: {experiment.experiment_id}")
    print(f"  Lifecycle stage: {experiment.lifecycle_stage}")
    print(f"  Artifact location: {experiment.artifact_location}")
except Exception as e:
    print(f"✗ Error accessing experiment: {e}")
    raise

## 2. Data Extraction

In [ ]:
# Fetch all runs from the experiment
print("Fetching all runs from the experiment...")
runs = client.search_runs(
    experiment_ids=[EXPERIMENT_ID],
    max_results=10000,  # Adjust if you expect more runs
)

print(f"Found {len(runs)} runs in the experiment")

if len(runs) == 0:
    print("No runs found in this experiment.")
else:
    print(f"Run status breakdown:")
    status_counts = {}
    for run in runs:
        status = run.info.status
        status_counts[status] = status_counts.get(status, 0) + 1

    for status, count in status_counts.items():
        print(f"  {status}: {count} runs")

In [ ]:
df

In [ ]:
rows = pyd.map_(
    runs,
    lambda r: {
        "group_id": pyd.get(r, "data.tags.group_id"),
        metric_name: pyd.get(r, metric_data_path),
        design_choic_name: pyd.get(r, design_choice_data_path),
        "run_status": pyd.get(r, "info.status"),
    },
)
df = pd.DataFrame(rows)
df = df.sort_values(by=["group_id", design_choic_name])
df["rank"] = df.groupby("group_id")[metric_name].rank(ascending=False)

In [ ]:
# filter out groups in which some runs failed
df = df.groupby("group_id").filter(lambda x: (x["run_status"] == "FINISHED").all())

In [ ]:
print(df.groupby(design_choic_name)["rank"].agg(["mean", "count"]).to_markdown())

In [ ]:
sns.violinplot(df, hue=design_choic_name, y="rank")